In [71]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import stim
print(stim.__version__)

import matplotlib.pyplot as plt
import numpy as np
import math

from ldpc import bp_decoder, bposd_decoder
import time
from src.utils import rank
from src.codes_q import create_bivariate_bicycle_codes, create_circulant_matrix
from src.build_circuit import build_circuit, dem_to_check_matrices
from src import osd_window

# [[72,12,6]]
# code, A_list, B_list = create_bivariate_bicycle_codes(6, 6, [3], [1,2], [1,2], [3])
# d = 6

# [[90,8,10]]
# code, A_list, B_list = create_bivariate_bicycle_codes(15, 3, [9], [1,2], [2,7], [0])
# d = 10

# [[108,8,10]]
# code, A_list, B_list = create_bivariate_bicycle_codes(9, 6, [3], [1,2], [1,2], [3])
# d = 10

# [[144,12,12]]
code, A_list, B_list = create_bivariate_bicycle_codes(12, 6, [3], [1,2], [1,2], [3])
d = 12

# [[288,12,18]]
# code, A_list, B_list = create_bivariate_bicycle_codes(12, 12, [3], [2,7], [1,2], [3])
# d = 18

# [[360,12,<=24]]
# code, A_list, B_list = create_bivariate_bicycle_codes(30, 6, [9], [1,2], [25,26], [3])

# [[756,16,<=34]]
# code, A_list, B_list = create_bivariate_bicycle_codes(21,18, [3], [10,17], [3,19], [5])

print(code.name)

1.13.dev1701377008
BB_n288_k12


This notebook tries to reproduce the result in [1] Figure 3.

Toggle the code above and change the `num_repeat` parameter to the code distance in the `build_circuit` function below. This function implements the circuit in Figure 7 / Table 5 of [1].

Following the Stim style of circuit creation:
- all data qubits are initialized into $|0\rangle$ if in the z-basis or $|+\rangle$ if in the x-basis.
- the first round is an encoding round (using the same syndrome measurement circuit), detectors are put onto the z-checks if in the z-basis or onto x-checks if in the x-basis. Detectors can NOT be put onto both types of checks in the encoding round.
- afterwards, syndrome measurement rounds are repeated `num_repeat-1` times, detectors are the XORed results of current z/x-checks and those from the previous round if in the z/x-basis. Detectors can be put onto both types of checks when setting `use_both` to `True`. It is not recommended to do so, as the detector error model check matrix will otherwise be too large. In other words, only independent X and Z decoding (`use_both=False`) is implemented in this repo.
- the data qubits are directly measured in the z/x-basis, and a noiseless syndrome measurement results are calculated using these data qubit measurement results. Again, these results are XORed with the previous round. This mimics the behavior of memory experiments (cf. surface code). This round is a simulation trick, as data qubits are never collapsed in real-time experiments (e.g. before a non-Clifford gate). An alternative implementation of this round is to still use syndrome measurement circuit but remove noise from CNOT gates.
- observables are the logical operators.
- Currently only one identity gate on L/R data is implemented as before-round-depolarization, performance does not accord with [1] if put two (cf. Table 5 [1]). Ancilla measurement and re-initialization is merged into `MR` in Stim.

The fault mechanism combination described in Section 6 of [1] is handled by Stim `detector_error_model` and the glue code `dem_to_check_matrices` was adapted from PyMatching.

[1] High-threshold and low-overhead fault-tolerant quantum memory

In [72]:
def decode(p, num_repeat, num_shots=10000, osd_order=10, shorten=False):  # whether use my heuristic OSD on shortened PCM)
    circuit = build_circuit(code, A_list, B_list, 
                            p=p, # physical error rate
                            num_repeat=num_repeat, # usually set to code distance
                            z_basis=True,   # whether in the z-basis or x-basis
                            use_both=False, # whether use measurement results in both basis to decode one basis
                           )
    dem = circuit.detector_error_model()
    chk, obs, priors, col_dict = dem_to_check_matrices(dem, return_col_dict=True)
    num_row, num_col = chk.shape
    chk_row_wt = np.sum(chk, axis=1)
    chk_col_wt = np.sum(chk, axis=0)
    print(f"check matrix shape {chk.shape}, max (row, column) weight ({np.max(chk_row_wt)}, {np.max(chk_col_wt)}),",
          f"min (row, column) weight ({np.min(chk_row_wt)}, {np.min(chk_col_wt)})")
    if not shorten:
        bpd = bposd_decoder(
            chk, # the parity check matrix
            error_rate=p, # does not matter as channel_prob is set
            channel_probs=priors, # assign error_rate to each qubit. This will override "error_rate" input variable
            max_iter=10000, # the maximum number of iterations for BP
            bp_method="minimum_sum_log", # messages are not clipped, may have numerical issues
            ms_scaling_factor=1.0, # min sum scaling factor. If set to zero the variable scaling factor method is used
            osd_method="osd_cs", # the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
            osd_order=osd_order, # the osd search depth, not specified in [1]
            input_vector_type="syndrome", # "received_vector"
        )
    else: # see Sliding Window OSD.ipynb for more detail
        bpd = osd_window(
            chk,
            channel_probs=priors,
            pre_max_iter=16, # BP preprocessing on original PCM
            post_max_iter=1000, # BP on shortened PCM
            ms_scaling_factor=1.0,
            new_n=None, # if set to None, 2*num_row columns will be kept
            osd_method="osd_cs",
            osd_order=10
        )

    start_time = time.perf_counter()
    dem_sampler: stim.CompiledDemSampler = dem.compile_sampler()
    det_data, obs_data, err_data = dem_sampler.sample(shots=num_shots, return_errors=False, bit_packed=False)
    print("detector data shape", det_data.shape)
    print("observable data shape", obs_data.shape)
    end_time = time.perf_counter()
    print(f"Stim: noise sampling for {num_shots} shots, elapsed time:", end_time-start_time)

    num_err = 0
    num_flag_err = 0
    start_time = time.perf_counter()
    for i in range(num_shots):
        e_hat = bpd.decode(det_data[i])
        num_flag_err += ((chk @ e_hat + det_data[i]) % 2).any()
        ans = (obs @ e_hat + obs_data[i]) % 2
        num_err += ans.any()
    end_time = time.perf_counter()
    print("Elapsed time:", end_time-start_time)
    print(f"Flagged Errors: {num_flag_err}/{num_shots}") # expect 0 for OSD
    print(f"Logical Errors: {num_err}/{num_shots}")
    p_l = num_err / num_shots
    p_l_per_round = 1-(1-p_l) ** (1/num_repeat)
    print("Logical error per round:", p_l_per_round)

In [6]:
decode(p=0.004, num_repeat=d, num_shots=10000, shorten=True)

check matrix shape (936, 8784), max (row, column) weight (35, 6), min (row, column) weight (16, 2)
detector data shape (10000, 936)
observable data shape (10000, 12)
Stim: noise sampling for 10000 shots, elapsed time: 0.023050088435411453
Elapsed time: 428.5726739112288
Flagged Errors: 0/10000
Logical Errors: 90/10000
Logical error per round: 0.0007531116566323881


In [382]:
decode(p=0.004, num_repeat=d, num_shots=10000)

check matrix shape (936, 8784), max (row, column) weight (35, 6), min (row, column) weight (16, 2)
detector data shape (10000, 936)
observable data shape (10000, 12)
Stim: noise sampling for 10000 shots, elapsed time: 0.018485471606254578
Elapsed time: 1657.7230640873313
Flagged Errors: 0/10000
Logical Errors: 76/10000
Logical error per round: 0.0006355502160568793


In [384]:
decode(p=0.003, num_repeat=d, num_shots=100000)

check matrix shape (936, 8784), max (row, column) weight (35, 6), min (row, column) weight (16, 2)
detector data shape (100000, 936)
observable data shape (100000, 12)
Stim: noise sampling for 100000 shots, elapsed time: 0.2078531989827752
Elapsed time: 4199.123198093846
Flagged Errors: 0/100000
Logical Errors: 77/100000
Logical error per round: 6.418932329932403e-05
